In [1]:
import pandas as pd

In [2]:
# Read datasets

df_bookings = pd.read_csv("datasets/f_bookings.csv")
df_date = pd.read_csv("datasets/d_date.csv")
df_hotels = pd.read_csv("datasets/d_hotels.csv")
df_rooms = pd.read_csv("datasets/d_rooms.csv")
df_agg_bookings = pd.read_csv("datasets/f_agg_bookings.csv")

### Explore aggregate bookings

In [3]:
df_bookings.head(4)

,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized
0,May012216558RT11,16558,27-04-22,1/5/2022,2/5/2022,-3.0,RT1,direct online,1.0,Checked Out,10010,10010
1,May012216558RT12,16558,30-04-22,1/5/2022,2/5/2022,2.0,RT1,others,NaN,Cancelled,9100,3640
2,May012216558RT13,16558,28-04-22,1/5/2022,4/5/2022,2.0,RT1,logtrip,5.0,Checked Out,9100000,9100
3,May012216558RT14,16558,28-04-22,1/5/2022,2/5/2022,-2.0,RT1,others,NaN,Cancelled,9100,3640


In [4]:
df_bookings.shape

(134590, 12)

In [5]:
df_bookings.describe()

,property_id,no_guests,ratings_given,revenue_generated,revenue_realized
count,134590.000000,134587.000000,56683.000000,1.345900e+05,134590.000000
mean,18061.113493,2.036170,3.619004,1.537805e+04,12696.123256
std,1093.055847,1.034885,1.235009,9.303604e+04,6928.108124
min,16558.000000,-17.000000,1.000000,6.500000e+03,2600.000000
25%,17558.000000,1.000000,3.000000,9.900000e+03,7600.000000
50%,17564.000000,2.000000,4.000000,1.350000e+04,11700.000000
75%,18563.000000,2.000000,5.000000,1.800000e+04,15300.000000
max,19563.000000,6.000000,5.000000,2.856000e+07,45220.000000


In [ ]:
df_bookings.booking_platform.value_counts().plot(kind="bar")

In [ ]:
df_agg_bookings.head(3)

In [ ]:
df_agg_bookings.property_id.unique()

In [ ]:
df_agg_bookings.groupby('property_id')['successful_bookings'].sum()

In [ ]:
df_agg_bookings[df_agg_bookings.successful_bookings > df_agg_bookings.capacity]

In [ ]:
df_agg_bookings[df_agg_bookings['capacity'] == df_agg_bookings['capacity'].max()]

### Data Cleaning

In [ ]:
df_bookings.describe()

**Clean invalid quests**

In [ ]:
df_bookings = df_bookings[df_bookings.no_guests > 0]

In [ ]:
df_bookings.describe()

**Outlier removal in revenue generated**

In [ ]:
higher_limit = df_bookings.revenue_generated.mean() + 3*df_bookings.revenue_generated.std()
higher_limit

In [ ]:
lower_limit = df_bookings.revenue_generated.mean() - 3*df_bookings.revenue_generated.std()
lower_limit
df_bookings.shape

In [ ]:
df_bookings[df_bookings.revenue_generated <= 0]

In [ ]:
df_bookings = df_bookings[df_bookings.revenue_generated <= higher_limit]
df_bookings.shape

In [ ]:
df_bookings.revenue_realized.describe()

In [ ]:
higher_limit = df_bookings.revenue_realized.mean() + 3*df_bookings.revenue_realized.std()
higher_limit

In [ ]:
df_bookings[df_bookings.revenue_realized < 0]


In [ ]:
df_bookings[df_bookings.revenue_realized>higher_limit]

In [ ]:
df_bookings[df_bookings.room_category=="RT4"].revenue_realized.describe()

In [ ]:
# mean + 3*standard deviation
23439+3*9048

In [ ]:
# 50583 is higher than 45220 so its reasonable that there are no outliers now

**Replace null values in aggregate bookings with mean()**

In [ ]:
#df_agg_bookings[df_agg_bookings.isnull().any(axis=1)]
df_agg_bookings.isnull().sum()

In [ ]:
df_agg_bookings.fillna(df_agg_bookings.capacity.mean(), inplace=True)

In [ ]:
df_agg_bookings.isnull().sum()

In [ ]:
df_agg_bookings = df_agg_bookings[df_agg_bookings.successful_bookings <= df_agg_bookings.capacity]

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings.apply(lambda x: x['successful_bookings']/x['capacity'], axis=1)

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings['occ_pct'].apply(lambda x: round(x*100, 2))

In [ ]:
df_agg_bookings

In [ ]:
#There are various types of data transformations

#Creating new columns
#Normalization
#Merging data
#Aggregation

### Insights Generation

**Average occupancy rate in each of the room categories**

In [ ]:
df_agg_bookings.value_counts()

In [ ]:
df_rooms.value_counts()

In [ ]:
df_merged = pd.merge(df_agg_bookings, df_rooms, left_on="room_category", right_on="room_id")
df_merged.head(4)

In [ ]:
df_merged.drop("room_id", axis=1, inplace=True)
df_merged.head(4)

In [ ]:
df_merged.groupby("room_category")["occ_pct"].mean()

In [ ]:
print(df_merged[df_merged.room_class == "Standard"].occ_pct.mean())

**Average occupancy rate per city**

In [ ]:
df_hotels.value_counts()

In [ ]:
df_merged = pd.merge(df_merged, df_hotels, on="property_id")
df_merged.head(4)

In [ ]:
df_merged.groupby("city")["occ_pct"].mean()

**When was occupancy better? Weekday or Weekend?**

In [ ]:
df_date.head(4)

In [ ]:
df_merged = pd.merge(df_merged, df_date, left_on="check_in_date", right_on="date")

In [ ]:
df_merged.groupby("day_type")["occ_pct"].mean().round()

**In the month of June, what is the occupancy for different cities**

In [ ]:
df_merged.value_counts()

In [ ]:
df_date.value_counts()

In [ ]:
df_june = df_merged[df_merged["mmm yy"] =="Jul 22"]
df_june.shape

In [ ]:
df_june.groupby("city")["occ_pct"].mean().round(2).sort_values(ascending=False)

**New data for the month of august. Append that to existing data**

In [ ]:
df_august = pd.read_csv("datasets/new_data_august.csv")
df_august

In [ ]:
df_latest = pd.concat([df_merged, df_august], ignore_index=True, axis=0)
df_latest.tail(10)

**Print revenue realized per city**

In [ ]:
df_bookings.head(4)

In [ ]:
df_hotels.head(4)

In [ ]:
df_bookings_hotels = pd.merge(df_bookings, df_hotels, on="property_id")
df_bookings_hotels.head(4)

In [ ]:
df_bookings_hotels.groupby("city")["revenue_realized"].sum()

**Print month by month revenue**

In [ ]:
df_date["date"].unique()

In [ ]:
df_date["mmm yy"].unique()

In [ ]:
pd.merge(df_bookings_hotels, df_date, left_on="check_in_date", right_on="date")

In [ ]:
df_bookings_hotels["check_in_date"] = pd.to_datetime(df_bookings_hotels["check_in_date"], format='mixed')

In [ ]:
df_date["date"] = pd.to_datetime(df_date["date"])
df_date.head(4)

In [ ]:
df_bookings_hotels = pd.merge(df_bookings_hotels, df_date, left_on="check_in_date", right_on="date")
df_bookings_hotels.head(4)

In [ ]:
df_bookings_hotels.groupby('mmm yy')['revenue_generated'].sum()

**Print revenue realized per hotel type**

In [ ]:
df_bookings_hotels.groupby('category')['revenue_realized'].sum()

**Print average rating per city**

In [ ]:
df_bookings_hotels = df_bookings_hotels[df_bookings_hotels.ratings_given.notnull()]

In [ ]:
df_bookings_hotels.groupby('city')['ratings_given'].mean().round(2)

**Print a pie chart of revenue realized per booking platform**

In [ ]:
g = df_bookings_hotels.groupby('booking_platform')['revenue_realized'].sum()
g

In [ ]:
g.plot(kind="pie")